# Handling A Long Context via `TransformChatHistory`

<div class="alert alert-warning" role="alert">
  <strong>Deprecation Notice:</strong> <code>TransformChatHistory</code> is no longer supported. Please use <code>TransformMessages</code> as the new standard method. For the latest examples, visit the notebook at <a href="https://github.com/microsoft/autogen/blob/main/notebook/agentchat_transform_messages.ipynb" target="_blank">notebook/agentchat_transform_messages.ipynb</a>.
</div>

This notebook illustrates how you can use the `TransformChatHistory` capability to give any `Conversable` agent an ability to handle a long context. 

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

In [2]:
import os
import autogen
from autogen.agentchat.contrib.capabilities import context_handling

In [3]:
llm_config = {
    "config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ.get("OPENAI_API_KEY")}],
}

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

To add this ability to any agent, define the capability and then use `add_to_agent`.

In [4]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config=llm_config,
)


# Instantiate the capability to manage chat history
manage_chat_history = context_handling.TransformChatHistory(max_tokens_per_message=50, max_messages=10, max_tokens=1000)
# Add the capability to the assistant
manage_chat_history.add_to_agent(assistant)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=10,
)

user_proxy.initiate_chat(assistant, message="plot and save a graph of x^2 from -10 to 10")

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
assistant (to user_proxy):

To plot and save a graph of the function x^2 from -10 to 10, you can use the matplotlib library in Python. Here is the code:

```python
import matplotlib.pyplot as plt
import numpy as np

# Generate x values from -10 to 10
x = np.linspace(-10, 10, 100)

# Calculate corresponding y values (x^2)
y = x**2

# Create the plot
plt.plot(x, y)

# Add labels and title
plt.xlabel('x')
plt.ylabel('y')
plt.title('Plot of x^2')

# Save the plot as a file
plt.savefig('x_squared_plot.png')

# Show the plot
plt.show()
```

This code will create a plot of the function x^2 and save it as "x_squared_plot.png" in the current directory. Make sure you have the matplotlib library installed before running this code.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (

ChatResult(chat_history=[{'content': 'plot and save a graph of x^2 from -10 to 10', 'role': 'assistant'}, {'content': 'To plot and save a graph of the function x^2 from -10 to 10, you can use the matplotlib library in Python. Here is the code:\n\n```python\nimport matplotlib.pyplot as plt\nimport numpy as np\n\n# Generate x values from -10 to 10\nx = np.linspace(-10, 10, 100)\n\n# Calculate corresponding y values (x^2)\ny = x**2\n\n# Create the plot\nplt.plot(x, y)\n\n# Add labels and title\nplt.xlabel(\'x\')\nplt.ylabel(\'y\')\nplt.title(\'Plot of x^2\')\n\n# Save the plot as a file\nplt.savefig(\'x_squared_plot.png\')\n\n# Show the plot\nplt.show()\n```\n\nThis code will create a plot of the function x^2 and save it as "x_squared_plot.png" in the current directory. Make sure you have the matplotlib library installed before running this code.', 'role': 'user'}, {'content': 'exitcode: 0 (execution succeeded)\nCode output: \nFigure(640x480)\n', 'role': 'assistant'}, {'content': "Great! 

## Why is this important?
This capability is especially useful if you expect the agent histories to become exceptionally large and exceed the context length offered by your LLM.
For example, in the example below, we will define two agents -- one without this ability and one with this ability.

The agent with this ability will be able to handle longer chat history without crashing.

In [5]:
assistant_base = autogen.AssistantAgent(
    "assistant",
    llm_config=llm_config,
)

assistant_with_context_handling = autogen.AssistantAgent(
    "assistant",
    llm_config=llm_config,
)
# suppose this capability is not available
manage_chat_history = context_handling.TransformChatHistory(max_tokens_per_message=50, max_messages=10, max_tokens=1000)
manage_chat_history.add_to_agent(assistant_with_context_handling)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    max_consecutive_auto_reply=2,
)

# suppose the chat history is large
# Create a very long chat history that is bound to cause a crash
# for gpt 3.5
long_history = []
for i in range(1000):
    # define a fake, very long message
    assitant_msg = {"role": "assistant", "content": "test " * 1000}
    user_msg = {"role": "user", "content": ""}

    assistant_base.send(assitant_msg, user_proxy, request_reply=False, silent=True)
    assistant_with_context_handling.send(assitant_msg, user_proxy, request_reply=False, silent=True)
    user_proxy.send(user_msg, assistant_base, request_reply=False, silent=True)
    user_proxy.send(user_msg, assistant_with_context_handling, request_reply=False, silent=True)

try:
    user_proxy.initiate_chat(assistant_base, message="plot and save a graph of x^2 from -10 to 10", clear_history=False)
except Exception as e:
    print("Encountered an error with the base assistant")
    print(e)
    print("\n\n")

try:
    user_proxy.initiate_chat(
        assistant_with_context_handling, message="plot and save a graph of x^2 from -10 to 10", clear_history=False
    )
except Exception as e:
    print(e)

user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
Encountered an error with the base assistant
Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 1009487 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}



user_proxy (to assistant):

plot and save a graph of x^2 from -10 to 10

--------------------------------------------------------------------------------
Truncated 1991 messages. Reduced from 2001 to 10.
Truncated 1000800 tokens. Tokens reduced from 1001015 to 215
assistant (to user_proxy):

Here's the Python code to plot and save a graph of x^2 from -10 to 10:

```python
# filename: plot_graph.py

import matplotlib.pyplot as plt
import numpy as np

# Generate x values from -10 to 10
x = np.linspace(-10, 10, 100)

